In [1]:
%load_ext autoreload
%autoreload 2

import tabulate
from omegaconf import OmegaConf
import wandb_util.wandb_util as wbu
from scripts.wandb_experiments.benchmark import BenchmarkConfig
import multiprocessing as mp
mp.set_start_method('spawn', force=True)

In [3]:
from scripts.wandb_experiments.benchmark import GeometryAndPrompts
from text3d2video.util import concat_lists

cat_prompts = ["Metalic Cat Statue"]

mv_cat = GeometryAndPrompts(
    "mv_cat_statue_25:latest",
    "cat_statue_mv:latest",
    "cat_statue_mv:latest",
    ["Metalic Cat Statue"],
    1,
)

mv_helmet = GeometryAndPrompts(
    "mv_helmet_25:latest",
    "mv_helmet:latest",
    "mv_helmet:latest",
    ["Shiny Helmet"],
    1,
)

ymca = GeometryAndPrompts(
    "ymca_20:latest",
    "human_mv:latest",
    "ymca_src:latest",
    ["Stormtrooper", "Deadpool"],
    1,
)

rumba = GeometryAndPrompts(
    "rumba_20:latest",
    "human_mv:latest",
    "rumba_src:latest",
    ["Stormtrooper"],
    1,
)


turn_around = GeometryAndPrompts(
    "catwalk_180_20:latest",
    "human_mv:latest",
    "catwalk_180_src:latest",
    ["Deadpool"],
    3,
)

geometries_and_promtps = [
    mv_cat,
    ymca,
    rumba,
    turn_around,
    mv_helmet,
]

scenes = concat_lists(s.to_scenes() for s in geometries_and_promtps)
print(tabulate.tabulate([s.tabulate_row() for s in scenes], headers="keys"))

animation_tag            texturing_tag         src_tag                 prompt                seed
-----------------------  --------------------  ----------------------  ------------------  ------
mv_cat_statue_25:latest  cat_statue_mv:latest  cat_statue_mv:latest    Metalic Cat Statue       0
ymca_20:latest           human_mv:latest       ymca_src:latest         Stormtrooper             0
ymca_20:latest           human_mv:latest       ymca_src:latest         Deadpool                 0
rumba_20:latest          human_mv:latest       rumba_src:latest        Stormtrooper             0
catwalk_180_20:latest    human_mv:latest       catwalk_180_src:latest  Deadpool                 0
catwalk_180_20:latest    human_mv:latest       catwalk_180_src:latest  Deadpool                 1
catwalk_180_20:latest    human_mv:latest       catwalk_180_src:latest  Deadpool                 2
mv_helmet_25:latest      mv_helmet:latest      mv_helmet:latest        Shiny Helmet             0


In [6]:
from scripts.wandb_experiments.benchmark import Method
from scripts.wandb_runs.render_texture import (
    RenderTextureConfig,
    render_texture_to_anim,
)
from scripts.wandb_runs.run_generative_rendering import (
    RunGenerativeRenderingConfig,
    run_generative_rendering,
)
from text3d2video.pipelines.generative_rendering_pipeline import (
    GenerativeRenderingConfig,
)
from text3d2video.pipelines.pipeline_utils import ModelConfig
from text3d2video.utilities.omegaconf_util import get_import_path

# GR base Config
base_gr = OmegaConf.structured(
    RunGenerativeRenderingConfig(
        prompt="",
        animation_tag="",
        generative_rendering=GenerativeRenderingConfig(num_keyframes=3),
        model=ModelConfig(),
        texture_tag=None,
        src_anim_tag=None,
    )
)

# ControlNet base Config
base_controlnet = OmegaConf.structured(
    RunGenerativeRenderingConfig(
        prompt="",
        animation_tag="",
        generative_rendering=GenerativeRenderingConfig(
            do_pre_attn_injection=False, do_post_attn_injection=False
        ),
        model=ModelConfig(),
        texture_tag=None,
        src_anim_tag=None,
    )
)

base_render = OmegaConf.structured(
    RenderTextureConfig(
        prompt="",
        animation_tag="",
        texture_tag="",
    )
)

methods = [
    Method("GR", get_import_path(run_generative_rendering), base_gr),
    Method("ControlNet", get_import_path(run_generative_rendering), base_controlnet),
    Method("Texture", get_import_path(render_texture_to_anim), base_render),
]

start_noises = [0, 0.2, 0.4]
for start_noise in start_noises:
    base_render_gr = OmegaConf.structured(
        RunGenerativeRenderingConfig(
            "",
            "",
            GenerativeRenderingConfig(),
            ModelConfig(),
            start_noise_level=start_noise,
            texture_tag="",
            src_anim_tag="",
        )
    )

    methods.append(
        Method(
            f"RenderThenGR-{start_noise}",
            get_import_path(run_generative_rendering),
            base_render_gr,
        )
    )

print(tabulate.tabulate([m.tabulate_row(with_config=False) for m in methods], headers="keys"))

name              fun
----------------  ------------------------
GR                run_generative_rendering
ControlNet        run_generative_rendering
Texture           render_texture_to_anim
RenderThenGR-0    run_generative_rendering
RenderThenGR-0.2  run_generative_rendering
RenderThenGR-0.4  run_generative_rendering


In [7]:
from scripts.wandb_experiments.benchmark import benchmark

config = BenchmarkConfig(scenes, methods)
config = OmegaConf.structured(config)

spec = benchmark(config)

In [9]:
from scripts.wandb_experiments.benchmark import benchmark
# 
wbu.sync_experiment(benchmark, config, "benchmark_new3")

Experiment: https://wandb.ai/romeu/diffusion-3D-features/groups/benchmark_new3/workspace

Would execute 0 new runs:

Would delete 2 outdated runs:
- Texture                        (d5u7wqcd)
- Texture                        (ktsmi5cj)

Do you want to continue? (y/n)
Aborting


In [10]:
from scripts.wandb_experiments.benchmark import split_runs

exp_name = "benchmark_new3"

runs = wbu.get_logged_runs(exp_name)
print(wbu.get_exp_url(exp_name))
texture_runs, video_gen_runs = split_runs(runs)

https://wandb.ai/romeu/diffusion-3D-features/groups/benchmark_new3/workspace


In [13]:
from text3d2video.util import group_into_array


def scene_key(run):
    config = OmegaConf.create(run.config)
    return f"{config.animation_tag}-{config.prompt}"

def method_key(run):
    return run.name.split("_")[0]

runs_grouped, labels = group_into_array(video_gen_runs, [method_key, scene_key])

In [16]:
from text3d2video.experiment_analysis import VideoTraces
from text3d2video.util import map_array

run_data = map_array(runs_grouped, VideoTraces.from_run, pbar=True)

































wandb:   20 of 20 files downloaded.  ..

wandb:   25 of 25 files downloaded.  ..

wandb:   20 of 20 files downloaded.  ..

wandb:   20 of 20 files downloaded.  ..

 83%|████████▎ | 30/36 [03:34<00:42,  7.14s/it]
wandb:   20 of 20 files downloaded.  

100%|██████████| 36/36 [00:43<00:00,  1.21s/it]


In [18]:
from text3d2video.clip_metrics import CLIPMetrics
from text3d2video.util import map_array

clip = CLIPMetrics()

# map_array(
    # run_data, partial(VideoTraces.compute_clip_metrics, model=clip), pbar=True
# )
# map_array(run_data, GeneratedVideoTraces.compute_uv_mse, pbar=True)

 83%|████████▎ | 30/36 [03:21<00:40,  6.71s/it]


In [29]:
from text3d2video.utilities.video_util import pil_frames_to_clip


In [31]:
from text3d2video.utilities.video_comparison import video_grid

clips_grid = map_array(run_data, lambda d: pil_frames_to_clip(d.frames))
vid = video_grid(clips_grid, y_labels=labels[0])
vid.write_videofile("outs/benchmark_with_tex.mp4")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Moviepy - Building video outs/benchmark_with_tex.mp4.
Moviepy - Writing video outs/benchmark_with_tex.mp4



  0%|          | 0/36 [02:50<?, ?it/s]

Moviepy - Done !
Moviepy - video ready outs/benchmark_with_tex.mp4


In [52]:
import tabulate


# table_rows = []
# values = []
table_rows = []

for row_i, row in enumerate(run_data):
    method = labels[0][row_i]

    # uv_mse = map_array(row, lambda d: d.uv_mse).mean()
    fc = map_array(row, lambda d: d.frame_consistency).mean()
    pf = map_array(row, lambda d: d.prompt_fidelity).mean()

    table_rows.append(
        {
            "Method": method,
            "UV MSE": None,
            "Frame Consistency": fc,
            "Prompt Fidelity": pf,
        }
    )

print(tabulate.tabulate(table_rows, headers="keys", tablefmt="markdown"))

Method            UV MSE      Frame Consistency    Prompt Fidelity
----------------  --------  -------------------  -----------------
ControlNet                             0.903655           0.29956
GR                                     0.966159           0.300501
RenderThenGR-0                         0.962151           0.299237
RenderThenGR-0.2                       0.95788            0.294666
RenderThenGR-0.4                       0.960727           0.293029
